## Analyze Data
This project aims at analyzing the content of an E-commerce database that lists purchases made by ∼4000 customers over a period of one year (from 2010/12/01 to 2011/12/09). Based on this analysis, I develop a model that allows to anticipate the purchases that will be made by a new customer, during the following year and this, from its first purchase.

You can download the data [here](https://www.kaggle.com/code/fabiendaniel/customer-segmentation/data)

## Load data

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("./data/data.csv", encoding="ISO-8859-1", dtype={'CustomerID': str,'InvoiceID': str})

In [3]:
data.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom


In [4]:
data.tail(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,12/9/2011 12:50,4.95,12680,France


In [5]:
data.sample(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
532763,580997,22692,DOORMAT WELCOME TO OUR HOME,1,12/6/2011 17:23,8.25,15023,United Kingdom
244232,558475,23197,SKETCHBOOK MAGNETIC SHOPPING LIST,1,6/29/2011 15:58,3.29,NaN,United Kingdom
512462,579512,23002,TRAVEL CARD WALLET SKULLS,2,11/29/2011 16:47,0.83,NaN,United Kingdom
76651,542634,84879,ASSORTED COLOUR BIRD ORNAMENT,8,1/31/2011 10:42,1.69,15448,United Kingdom
422298,573076,23581,JUMBO BAG PAISLEY PARK,10,10/27/2011 14:05,2.08,13488,United Kingdom
212653,555500,21688,SILVER PLATE CANDLE BOWL SMALL,6,6/3/2011 15:32,2.95,15508,United Kingdom
158698,550310,22919,HERB MARKER MINT,24,4/17/2011 11:04,0.65,15985,United Kingdom
173588,551718,22183,CAKE STAND VICTORIAN FILIGREE MED,2,5/3/2011 16:06,5.79,NaN,United Kingdom
397923,571214,22386,JUMBO BAG PINK POLKADOT,5,10/14/2011 12:56,2.08,14390,United Kingdom
300630,C563207,20781,GOLD EAR MUFF HEADPHONES,-1,8/14/2011 12:10,5.49,17613,United Kingdom


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  object 
 7   Country      541909 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 33.1+ MB


In [7]:
data.describe()

,Quantity,UnitPrice
count,541909.000000,541909.000000
mean,9.552250,4.611114
std,218.081158,96.759853
min,-80995.000000,-11062.060000
25%,1.000000,1.250000
50%,3.000000,2.080000
75%,10.000000,4.130000
max,80995.000000,38970.000000


In [8]:
data.describe(include=object)

,InvoiceNo,StockCode,Description,InvoiceDate,CustomerID,Country
count,541909,541909,540455,541909,406829,541909
unique,25900,4070,4223,23260,4372,38
top,573585,85123A,WHITE HANGING HEART T-LIGHT HOLDER,10/31/2011 14:41,17841,United Kingdom
freq,1114,2313,2369,1114,7983,495478


In [9]:
data.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

## Fill NaNs

### Try to use stock code to fill Description

In [10]:
stock_descriptions = data[["StockCode", "Description"]]

stock_descriptions.sample(10)

,StockCode,Description
169842,62018,SOMBRERO
63430,22431,WATERING CAN BLUE ELEPHANT
447051,23203,JUMBO BAG VINTAGE DOILY
408805,47566,PARTY BUNTING
398563,23076,ICE CREAM SUNDAE LIP GLOSS
322893,21094,SET/6 RED SPOTTY PAPER PLATES
30689,22734,SET OF 6 RIBBONS VINTAGE CHRISTMAS
496205,22643,SET OF 4 NAPKIN CHARMS LEAVES
15430,22740,POLKADOT PEN
223543,22139,RETROSPOT TEA SET CERAMIC 11 PC


In [11]:
stock_descriptions.describe()

,StockCode,Description
count,541909,540455
unique,4070,4223
top,85123A,WHITE HANGING HEART T-LIGHT HOLDER
freq,2313,2369


Stock Code => Description

In [12]:
stock_descriptions[stock_descriptions.StockCode == "85123A"].sample(10)

,StockCode,Description
162810,85123A,WHITE HANGING HEART T-LIGHT HOLDER
34056,85123A,WHITE HANGING HEART T-LIGHT HOLDER
225639,85123A,WHITE HANGING HEART T-LIGHT HOLDER
179292,85123A,WHITE HANGING HEART T-LIGHT HOLDER
14087,85123A,WHITE HANGING HEART T-LIGHT HOLDER
264875,85123A,WHITE HANGING HEART T-LIGHT HOLDER
275887,85123A,WHITE HANGING HEART T-LIGHT HOLDER
242000,85123A,WHITE HANGING HEART T-LIGHT HOLDER
83792,85123A,WHITE HANGING HEART T-LIGHT HOLDER
329145,85123A,WHITE HANGING HEART T-LIGHT HOLDER


In [13]:
stocks_nans = stock_descriptions[stock_descriptions.Description.isna()]

stocks_nans.sample(10)

,StockCode,Description
222615,37479B,NaN
42500,21784,NaN
205712,16049,NaN
55270,22323,NaN
242439,47013C,NaN
148111,21171,NaN
167145,22777,NaN
172868,85226C,NaN
75257,85119,NaN
147827,79336,NaN


In [14]:
stocks_no_nans = stock_descriptions.drop(stocks_nans.index)

stocks_no_nans.sample(10)

,StockCode,Description
126014,22193,RED DINER WALL CLOCK
432523,84947,ANTIQUE SILVER TEA GLASS ENGRAVED
107039,22262,FELT EGG COSY CHICKEN
43831,84992,72 SWEETHEART FAIRY CAKE CASES
167126,22087,PAPER BUNTING WHITE LACE
72883,21068,VINTAGE BILLBOARD LOVE/HATE MUG
382427,22334,DINOSAUR PARTY BAG + STICKER SET
477333,85071B,RED CHARLIE+LOLA PERSONAL DOORSIGN
155061,22630,DOLLY GIRL LUNCH BOX
172729,21121,SET/10 RED POLKADOT PARTY CANDLES


In [15]:
stcok_intersection = np.intersect1d(stocks_nans.StockCode, stocks_no_nans.StockCode)

len(stcok_intersection)

848

We can fill 848 on 960 absents descriptions

In [16]:
len(stocks_nans.StockCode.unique())

960

Compute unique StockCode -> Description dict

In [17]:
stock_desc_dict: pd.DataFrame = stocks_no_nans.groupby("StockCode").first().reset_index()

stock_desc_dict.head()

,StockCode,Description
0,10002,INFLATABLE POLITICAL GLOBE
1,10080,GROOVY CACTUS INFLATABLE
2,10120,DOGGY RUBBER
3,10123C,HEARTS WRAPPING TAPE
4,10124A,SPOTS ON RED BOOKCOVER TAPE


In [18]:
stock_desc_dict = dict(zip(stock_desc_dict.StockCode, stock_desc_dict.Description))

len(stock_desc_dict)

3958

In [19]:
def fill_description(x):
    x.Description = stock_desc_dict.get(x.StockCode)
    return x

In [20]:
stocks_nans = stocks_nans.apply(lambda x: fill_description(x), axis=1)

In [21]:
stocks_nans.sample(10)

,StockCode,Description
131703,17011F,ORIGAMI SANDLEWOOD INCENSE+FLOWER
243333,22164,STRING OF STARS CARD HOLDER
136886,21051,RIBBONS PURSE
204600,20664,TROPICAL HOLIDAY PURSE
75232,22143,None
468231,22687,DOORMAT CHRISTMAS VILLAGE
136095,21445,12 PINK ROSE PEG PLACE SETTINGS
172919,21854,None
382666,37370,RETRO COFFEE MUGS ASSORTED
363135,22409,MONEY BOX BISCUITS DESIGN


In [22]:
stocks_nans.describe()

,StockCode,Description
count,1454,1342
unique,960,846
top,35965,FOLKART HEART NAPKIN RINGS
freq,10,10


In [23]:
stock_descriptions = pd.concat([stocks_nans, stocks_no_nans])

stock_descriptions.shape

(541909, 2)

In [24]:
data.shape

(541909, 8)

In [25]:
for column in stock_descriptions.columns:
    data[column] = stock_descriptions[column]


data.describe(include=object)

,InvoiceNo,StockCode,Description,InvoiceDate,CustomerID,Country
count,541909,541909,541797,541909,406829,541909
unique,25900,4070,4223,23260,4372,38
top,573585,85123A,WHITE HANGING HEART T-LIGHT HOLDER,10/31/2011 14:41,17841,United Kingdom
freq,1114,2313,2369,1114,7983,495478


### Try to use invoice code to fill customer id

In [26]:
invoice_customers = data[["InvoiceNo", "CustomerID"]]

invoice_customers.sample(10)

,InvoiceNo,CustomerID
62249,541497,NaN
319522,564857,NaN
459491,575875,NaN
146634,549012,16394
458215,575760,15965
79726,543000,NaN
401895,571444,18179
119315,546541,14911
381217,569848,16316
216177,555835,14525


In [27]:
invoice_customers.isna().sum()

InvoiceNo          0
CustomerID    135080
dtype: int64

In [28]:
invoices_nans = invoice_customers[invoice_customers.CustomerID.isna()]

invoices_no_nans = invoice_customers.drop(invoices_nans.index)


len(invoices_nans), len(invoices_no_nans)

(135080, 406829)

In [29]:
invoice_intersection = np.intersect1d(invoices_no_nans.InvoiceNo, invoices_nans.InvoiceNo)

len(invoice_intersection)

0

Oups ! No data.

We will delete invoices without customer since our goal is to process on customer

## Delete Invoice without CustomerId

In [30]:
data = data.drop(data[data.CustomerID.isna()].index)

data.reset_index(drop=True, inplace=True)

data.sample(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
184236,559551,35965,FOLKART HEART NAPKIN RINGS,7,7/10/2011 16:18,0.79,16877,United Kingdom
306521,572186,23089,GLASS BON BON JAR,12,10/21/2011 10:55,1.65,16497,United Kingdom
295601,571216,22139,RETROSPOT TEA SET CERAMIC 11 PC,4,10/14/2011 13:06,4.95,16081,United Kingdom
387758,579571,85227,SET OF 6 3D KIT CARDS FOR KIDS,1,11/30/2011 11:48,0.85,16657,United Kingdom
58607,544077,22249,DECORATION WHITE CHICK MAGIC GARDEN,2,2/15/2011 15:04,0.85,14825,United Kingdom
222670,563920,22683,FRENCH BLUE METAL DOOR SIGN 8,3,8/21/2011 14:00,1.25,16407,United Kingdom
135273,553375,22982,PANTRY PASTRY BRUSH,12,5/16/2011 14:53,1.25,14911,EIRE
255033,567470,21213,PACK OF 72 SKULL CAKE CASES,1,9/20/2011 12:39,0.55,16624,United Kingdom
326103,574034,23368,SET 12 COLOUR PENCILS DOLLY GIRL,1,11/2/2011 12:45,0.65,17841,United Kingdom
55900,543674,21471,STRAWBERRY RAFFIA FOOD COVER,96,2/11/2011 11:28,2.95,16013,United Kingdom


In [31]:
data.isna().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

No NaNs 😃 👍

### Some cold exploration

> Customer Per Country

In [32]:
customer_per_country: pd.DataFrame = data[["CustomerID", "Country"]].groupby("Country")["CustomerID"].apply(lambda x: len(np.unique(x)))

customer_per_country.sort_values().tail()

Country
Belgium             25
Spain               31
France              87
Germany             95
United Kingdom    3950
Name: CustomerID, dtype: int64

In [33]:
import plotly.express as px

In [34]:
selected_customer_per_country = customer_per_country.sort_values().tail(10).reset_index()
selected_customer_per_country.columns = ["Country", "CustomerCount"]
selected_customer_per_country

,Country,CustomerCount
0,Austria,11
1,Finland,12
2,Italy,15
3,Portugal,19
4,Switzerland,21
5,Belgium,25
6,Spain,31
7,France,87
8,Germany,95
9,United Kingdom,3950


In [35]:
fig = px.pie(selected_customer_per_country, values='CustomerCount', names='Country', title='Top Customers Count per Country', height=600, width=800)
fig.show()

In [36]:
customer_per_country = customer_per_country.sort_values().reset_index()

customer_per_country.head(2)

,Country,CustomerID
0,European Community,1
1,Lebanon,1


In [37]:
customer_per_country.columns = ["Country", "CustomerCount"]

In [39]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [73]:
selected_customer_per_country = customer_per_country.copy()
selected_customer_per_country["Country"] = selected_customer_per_country[
    "Country"
].apply(lambda x: str(x).upper())
# " ".join(z.capitalize() for z in str(x).split())

selected_customer_per_country.head()

,Country,CustomerCount
0,EUROPEAN COMMUNITY,1
1,LEBANON,1
2,ICELAND,1
3,RSA,1
4,BRAZIL,1


In [83]:
print(selected_customer_per_country.Country.values)

['EUROPEAN COMMUNITY' 'LEBANON' 'ICELAND' 'RSA' 'BRAZIL' 'SAUDI ARABIA'
 'LITHUANIA' 'SINGAPORE' 'CZECH REPUBLIC' 'MALTA' 'BAHRAIN'
 'UNITED ARAB EMIRATES' 'EIRE' 'CANADA' 'USA' 'GREECE' 'ISRAEL'
 'UNSPECIFIED' 'POLAND' 'SWEDEN' 'JAPAN' 'CYPRUS' 'NETHERLANDS'
 'AUSTRALIA' 'DENMARK' 'CHANNEL ISLANDS' 'NORWAY' 'AUSTRIA' 'FINLAND'
 'ITALY' 'PORTUGAL' 'SWITZERLAND' 'BELGIUM' 'SPAIN' 'FRANCE' 'GERMANY'
 'UNITED KINGDOM']


In [96]:
example = pd.DataFrame([["0500000US01001", 100]], columns=["Country", "CustomerCount"])

In [97]:
fig = px.choropleth_mapbox(example,
    locations="Country", color="CustomerCount",
    mapbox_style="carto-positron",
    color_continuous_scale="Viridis",
    opacity=0.5,
    center = {"lat": 51.500000, "lon": -0.116667}
)
fig.update_layout(margin={"r":0, "t":0, "l":0, "b":0})
fig.show()

In [80]:
with open("test.json", "w") as file:
    json.dump(counties, file)

Signature:
px.choropleth_mapbox(
    data_frame=None,
    geojson=None,
    featureidkey=None,
    locations=None,
    color=None,
    hover_name=None,
    hover_data=None,
    custom_data=None,
    animation_frame=None,
    animation_group=None,
    category_orders=None,
    labels=None,
    color_discrete_sequence=None,
    color_discrete_map=None,
    color_continuous_scale=None,
    range_color=None,
    color_continuous_midpoint=None,
    opacity=None,
    zoom=8,
    center=None,
    mapbox_style=None,
    title=None,
    template=None,
    width=None,
    height=None,
) -> plotly.graph_objs._figure.Figure
Docstring:
    In a Mapbox choropleth map, each row of `data_frame` is represented by a
    colored region on a Mapbox map.
    
Parameters
----------
data_frame: DataFrame or array-like or dict
    This argument needs to be passed for column names (and not keyword
    names) to be used. Array-like and dict are tranformed internally to a
    pandas DataFrame. Optional: if missi